In [24]:
import os

from matplotlib import pyplot as plt
from PIL import Image
import cv2
import numpy as np

import sqlite3

from utils import *

In [25]:
with open("./0/points3D.txt", "r") as f:
    temp_pts = f.readlines()[3:]
pts3d = np.array([x.split(" ")[1:4] for x in temp_pts], dtype=np.float32)
rgb3d = np.array([x.split(" ")[4:7] for x in temp_pts], dtype=np.float32)/255

with open("./0/images.txt", "r") as f:
    temp_images = f.readlines()[4:][::2]
img_ids = np.array([x.split(" ")[0] for x in temp_images], dtype=np.int16)
poses = np.array([x.split(" ")[1:8] for x in temp_images], dtype=np.float32)

with open("./0/cameras.txt", "r") as f:
    temp_cameras = f.readlines()[-1]
cam_params = np.array(temp_cameras.split(" ")[-4:], dtype=np.float32)

con = sqlite3.connect("./database.db")
cur = con.cursor()
img_dict = dict()
for row in cur.execute("SELECT * FROM IMAGES"):
    img_dict[row[0]] = row[1]
con.close()

In [26]:
filenames = [img_dict[idx] for idx in img_ids]
images_path = "./images/"
image_paths = [os.path.join(images_path, fname) for fname in filenames]
images = [np.array(Image.open(path)) for path in image_paths]

In [27]:
from ultralytics import YOLO
import cv2
# Load a model
model = YOLO('./best.pt')  # load a custom model

In [28]:
masks = []
for image_path in image_paths:
    results = model(image_path, conf=0.2)

    mask = np.zeros((1280,720))
    for r in results:
        for box_info in range(len(r.boxes)):
            a = r.boxes[box_info].xyxy[0].tolist()

            bbox = np.int32(np.round(a))
            mask[bbox[1]:bbox[3], bbox[0]:bbox[2]] = 1
    
    masks.append(mask)


image 1/1 c:\Users\matth\CV Class\FrontAndBack\images\dataset1_front_1171.png: 640x384 30 apples, 7.9ms
Speed: 1.0ms preprocess, 7.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 c:\Users\matth\CV Class\FrontAndBack\images\dataset1_front_1.png: 640x384 44 apples, 4.5ms
Speed: 1.0ms preprocess, 4.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 c:\Users\matth\CV Class\FrontAndBack\images\dataset1_front_1021.png: 640x384 33 apples, 5.5ms
Speed: 1.0ms preprocess, 5.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 c:\Users\matth\CV Class\FrontAndBack\images\dataset1_front_1051.png: 640x384 37 apples, 4.5ms
Speed: 1.0ms preprocess, 4.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 c:\Users\matth\CV Class\FrontAndBack\images\dataset1_front_1081.png: 640x384 33 apples, 4.5ms
Speed: 1.5ms preprocess, 4.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384

In [29]:
from PIL import Image

mask_path = "./created_masks/"
if not os.path.exists(mask_path):
    os.makedirs(mask_path)

for mask,fname in zip(masks, filenames):
    im = Image.fromarray(np.uint8(255*mask), mode="L")
    im.save(os.path.join(mask_path, fname))